In [ ]:
import geopandas as gpd
import pandas as pd
from dash import dash, html, Input, Output
import dash_bootstrap_components as dbc

In [ ]:
# Need to add dbc.themes.BOOTSTRAP so styling of bootstrap components will work
app = dash.Dash(__name__)

In [ ]:
geo_df = gpd.read_file(r'./data/SC_Earthquake.geojson')

In [ ]:
geo_df['Event_Date'] = pd.to_datetime(geo_df.time, unit="ms") \
    .dt.tz_localize('UTC') \
    .dt.tz_convert('America/New_York').dt.date

geo_df['Event_Time'] = pd.to_datetime(geo_df.time, unit="ms") \
    .dt.tz_localize('UTC') \
    .dt.tz_convert('America/New_York').dt.time

# remove decimal portion of the seconds part of the time
for x in range(len(geo_df['Event_Time'])):
    geo_df.loc[x, 'Event_Time'] = geo_df['Event_Time'][x].replace(microsecond=0)

geo_df['Longitude'] = geo_df.geometry.x
geo_df['Latitude'] = geo_df.geometry.y
geo_df['Depth'] = geo_df.geometry.z

geo_df = geo_df[['mag', 'place', 'detail', 'felt', 'cdi', 'title', 'Latitude', 'Longitude', 'Depth', 'Event_Date', 'Event_Time']]
geo_df = geo_df.rename(columns={'mag': 'Mag', 'place': 'Place', 'detail': 'Url', 'felt': 'Felt',
                                'cdi': 'CDI', 'title': 'Title'})  # , 'geometry': 'Geometry'})
geo_df.Felt = geo_df.Felt.fillna(0).astype('int')
geo_df.CDI = geo_df.CDI.fillna(0).astype('float')


geo_df['Mag'] = geo_df.Mag.round(1)

geo_dff = geo_df.copy()


In [ ]:
geo_dff

In [ ]:
# geo_df.explore()

In [ ]:
app.layout = html.Div([
    html.Button('Table', id='table-but', n_clicks=0),
    html.Div(id='container-button-basic')
])

In [ ]:
@app.callback(
    Output('container-button-basic', 'children'),
    Input('table-but', 'n_clicks'))
def search_fi(n_clicks):
    if n_clicks > 0:
        table = dbc.Table.from_dataframe(geo_dff, striped=True, bordered=True, hover=True)
        return table

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False, port=8050)